In [10]:
import os
import gc
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import MobileNet, MobileNetV3Small, MobileNetV3Large
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve

In [ ]:
NUM_CLASSES = 2  # số nhãn

X = np.load('../SavedFeatures/X_f0.npy')
y = np.load('../SavedFeatures/y_f0.npy')

y_encoded = to_categorical(y, NUM_CLASSES)
split_index = int(0.8 * len(X))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y_encoded[:split_index], y_encoded[split_index:]

# Reshape data for MobileNet
X_train = np.expand_dims(X_train, axis=-1)
X_train = np.tile(X_train, (1, 1, 109))

X_val = np.expand_dims(X_val, axis=-1)
X_val = np.tile(X_val, (1, 1, 109)) 

del X,y
gc.collect()

In [9]:
input_shape = (109, 109, 1)

In [11]:
def mobilenet_v1(input_shape=(109, 109, 1), num_classes=2):
    base_model = MobileNet(input_shape=input_shape, weights=None, include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    output = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=output)

In [13]:
# Release GPU
tf.keras.backend.clear_session()

In [14]:
mbnv1 = mobilenet_v1(input_shape=(109, 109, 1), num_classes=2)
mbnv1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 109, 109, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 55, 55, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 55, 55, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 55, 55, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 55, 55, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 55, 55, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 55, 55, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 55, 55, 64)     │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 55, 55, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 27, 27, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 27, 27, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 27, 27, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 27, 27, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 27, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 27, 27, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 27, 27, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 27, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 27, 27, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 27, 27, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 27, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,374,882 (12.87 MB)

 Trainable params: 3,350,498 (12.78 MB)

 Non-trainable params: 24,384 (95.25 KB)

In [15]:
mbnv1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
# Train the Model
hist = mbnv1.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10


I0000 00:00:1741452759.987896  454911 service.cc:148] XLA service 0x7b5ab80037e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741452759.987914  454911 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2025-03-08 23:52:40.141018: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741452760.813880  454911 cuda_dnn.cc:529] Loaded cuDNN version 90300


 26/796 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5210 - loss: 0.9471 

I0000 00:00:1741452765.729999  454911 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


796/796 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.5617 - loss: 0.8528 - val_accuracy: 0.3998 - val_loss: 1.3546
Epoch 2/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6032 - loss: 0.7386 - val_accuracy: 0.6922 - val_loss: 0.6031
Epoch 3/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6466 - loss: 0.6823 - val_accuracy: 0.7032 - val_loss: 0.5739
Epoch 4/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6824 - loss: 0.6391 - val_accuracy: 0.7393 - val_loss: 0.5470
Epoch 5/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7165 - loss: 0.6016 - val_accuracy: 0.7535 - val_loss: 0.5280
Epoch 6/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7394 - loss: 0.5686 - val_accuracy: 0.7710 - val_loss: 0.5130
Epoch 7/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7527 - loss: 0.5445 - val_accuracy: 0.7654 - val_loss: 0.5203
Epoch 8/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7565 - loss: 0.5404 - val_accuracy: 0.7748 - va

In [ ]:
from sklearn.metrics import f1_score, roc_curve
import numpy as np

test_loss, test_accuracy = mbnv1.evaluate(X_val, y_val, verbose=0)

y_pred = mbnv1.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_val.argmax(axis=1)  # Chuyển đổi y_val sang nhãn lớp

f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================MobileNet - F0===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")

eers = []

# Tính EER 
for i in range(y_pred.shape[1]): 
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")

mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
===================MobileNet - F0===================
TAccuracy: 0.78046
F1-Score: 0.77956
Class 0: EER = 0.23926 at threshold 0.74577
Class 1: EER = 0.23883 at threshold 0.25591
EER: 0.23905


In [17]:
# Save model as weights in H5 format
mbnv1.save_weights("../SavedModel/F0_mbnv1.weights.h5")

In [18]:

del mbnv1, X_train, X_val, y_train, y_val
gc.collect()


6882